In [ ]:
!pip install pymysql
# load the necessary libraries
import codecs
import json
import matplotlib.pyplot as plt
import pandas as pd
import pymysql
import requests
import re
import time
import warnings
from bs4 import BeautifulSoup
from collections import Counter
from nltk.corpus import stopwords
from pandas import DataFrame
from tqdm import trange
from sqlalchemy import create_engine
import re
import datetime

     |████████████████████████████████| 51kB 7.0MB/s 


In [ ]:
# Define page num here:
MAX_PAGE_NUM = 100

Glassdoor

In [ ]:
titles =[]
locations =[]
companies =[]
summary =[]
timestamp =[]
salary=[]
salarybfcleaning =[]
daynum=[]
for i in range(0, MAX_PAGE_NUM):
  if i is not 0:
    print("Loading page " + str(i) + "...")
    url = 'https://www.glassdoor.com/Job/fintech-jobs-SRCH_KE0,7_IP'+str(i)+'.htm?includeNoSalaryJobs=true'# glassdoor
    hdr = {'User-agent': 'Mozilla/5.0 Chrome/58.0.3029.110'} 
    response = requests.get(url, headers = hdr)
    soup = BeautifulSoup(response.text, 'html.parser')
    divs = soup.find_all('ul', attrs={'data-test':'jlGrid'})
    for a in divs:
      # titles
      h2 = a.find_all('li', attrs={'data-test':'jobListing'})
      for b in h2:
        title = b.find('a', attrs={'data-test':'job-link'}).text
        titles.append(title)
      # company
      comset = a.find_all('div', attrs={'class':'d-flex justify-content-between align-items-start'})
      for c in comset:
        com = c.find('a', attrs={'class':' css-l2wjgv e1n63ojh0 jobLink'}).text
        companies.append(com)
      # location
      locset = a.find_all('div', {'class':'d-flex flex-wrap css-11d3uq0 e1rrn5ka2'})
      for d in locset:
        locations.append(d.text)
      # salary
      salset = a.find_all('div',{'class':'d-flex flex-wrap css-1i7b5bu e1rrn5ka1'})
      for e in salset:
        salarybfcleaning.append(e.text)
      for i in salarybfcleaning:
        if i.startswith("$") == False:
          salary.append(i.split('}')[-1])
        else:
          salary.append(i.split('(')[0])
      # date posted
      dateset = a.find_all('div',{'data-test':'job-age'})
      for f in dateset:
        daynum.append(f.text)
      # Actual date
      today = datetime.datetime.now()
      datesposted = []
      for i in daynum:
        if 'd' in i:
          dy = i.split('d')[0]
          numindt = datetime.timedelta(days= int(dy))
        else:
          try:
            dy = i.split('h')[0]
            numindt = datetime.timedelta(days= 0)
          except ImportError:
            getpass = default_getpass
          else:
            dy = i.split('h')[0]
            numindt = datetime.timedelta(days= 0)
        dateposted = (today-numindt).date()
        datesposted.append(dateposted)

Loading page 1...
Loading page 2...
Loading page 3...
Loading page 4...
Loading page 5...
Loading page 6...
Loading page 7...
Loading page 8...
Loading page 9...
Loading page 10...
Loading page 11...
Loading page 12...
Loading page 13...
Loading page 14...
Loading page 15...
Loading page 16...
Loading page 17...
Loading page 18...
Loading page 19...
Loading page 20...
Loading page 21...
Loading page 22...
Loading page 23...
Loading page 24...
Loading page 25...
Loading page 26...
Loading page 27...
Loading page 28...
Loading page 29...
Loading page 30...
Loading page 31...
Loading page 32...
Loading page 33...
Loading page 34...
Loading page 35...
Loading page 36...
Loading page 37...
Loading page 38...
Loading page 39...
Loading page 40...
Loading page 41...
Loading page 42...
Loading page 43...
Loading page 44...
Loading page 45...
Loading page 46...
Loading page 47...
Loading page 48...
Loading page 49...
Loading page 50...
Loading page 51...
Loading page 52...
Loading page 53...
Lo

In [ ]:
dict={'title':titles,'company':companies,'location':locations,'dateposted':datesposted}
df=pd.DataFrame(dict)
df

,title,company,location,dateposted
0,Director of Product Management (FinTech),Ando,.css-1ndif2q{color:#505863;font-weight:normal;...,2021-05-25
1,Entry Level Data Analyst - 4487,ION,"New York, NY30d+",2021-04-27
2,Equity Research Associate - Payments/Fintech,UBS,"New York, NY16d",2021-05-11
3,Corporate M&A Associate,Pollack Badawi Group,"Washington, DC30d+",2021-04-27
4,Travel Agent,Hopper,United States30d+,2021-04-27
...,...,...,...,...
837,Sr Financial Analyst FP&A,TMX Finance Family of Companies,"Carrollton, TX",2021-05-06
838,Propulsion & Fluid Systems Engineer,Bell Textron Inc.,"Fort Worth, TX",2021-05-06
839,Network Azure Cloud Architect (Fully Remote) -...,VTRAC Consulting Corporation,"Austin, TX",2021-05-12
840,System Engineer,"Neocis, Inc.","Miami, FL",2021-05-12


In [ ]:
df.to_csv('glassdoor-time.csv', index=False, encoding='utf-8')

In [ ]:
salarydf = DataFrame(salary)

In [ ]:
salarydf.to_csv('gdsal.csv', index=False, encoding='utf-8')

Indeed

In [ ]:
titles =[]
locations =[]
companies =[]
summary =[]
timestamp =[]
for i in range(0, MAX_PAGE_NUM):
  extension = ""
  if i is not 0:
    extension = "&start="+str(i*10)
    print("Loading page " + str(i) + "...")
    url = 'https://www.indeed.com/jobs?q=fintech&l=' + extension # indeed
    hdr = {'User-agent': 'Mozilla/5.0 Chrome/58.0.3029.110'} 
    response = requests.get(url, headers = hdr)
    soup = BeautifulSoup(response.text, 'html.parser')
    divs = soup.find_all('div', attrs={'class':'jobsearch-SerpJobCard unifiedRow row result'})
    for a in divs:
      # titles
      h2 = a.find_all('h2')
      for b in h2:
        title = b.find('a').text
        titles.append(title[1:])
      # companies
      comloc = a.find_all('a', {'data-tn-element':'companyName'})
      for c in comloc:
        companies.append(c.text[1:])
      # locations
      locationset = a.find_all('span',{'class':'location accessible-contrast-color-location'})
      for d in locationset:
        locations.append(d.text)
      # job summary
      jdset = a.find_all('ul',{'style':'list-style-type:circle;margin-top: 0px;margin-bottom: 0px;padding-left:20px;'})
      for e in jdset:
        summary.append(e.text[1:])
      # time stamp
      tmset = a.find_all('span', {'class':'date date-a11y'})
      for f in tmset:
        timestamp.append(f.text)
    # cleaning dates
    import re
    # striping days
    daynum =[]
    for i in timestamp:
      if any(map(str.isdigit, i)) == True:
        numfilter = filter(str.isdigit, i)
        num = "".join(numfilter)
        daynum.append(num)
      else:
        daynum.append(0)

    # Actual date
    import datetime
    today = datetime.datetime.now()
    datesposted = []
    for i in daynum:
      numindt = datetime.timedelta(days= int(i))
      dateposted = (today-numindt).date()
      datesposted.append(dateposted)

Loading page 1...
Loading page 2...
Loading page 3...
Loading page 4...
Loading page 5...
Loading page 6...
Loading page 7...
Loading page 8...
Loading page 9...
Loading page 10...
Loading page 11...
Loading page 12...
Loading page 13...
Loading page 14...
Loading page 15...
Loading page 16...
Loading page 17...
Loading page 18...
Loading page 19...
Loading page 20...
Loading page 21...
Loading page 22...
Loading page 23...
Loading page 24...
Loading page 25...
Loading page 26...
Loading page 27...
Loading page 28...
Loading page 29...
Loading page 30...
Loading page 31...
Loading page 32...
Loading page 33...
Loading page 34...
Loading page 35...
Loading page 36...
Loading page 37...
Loading page 38...
Loading page 39...
Loading page 40...
Loading page 41...
Loading page 42...
Loading page 43...
Loading page 44...
Loading page 45...
Loading page 46...
Loading page 47...
Loading page 48...
Loading page 49...
Loading page 50...
Loading page 51...
Loading page 52...
Loading page 53...
Lo

In [ ]:
dict={'title':titles,'dateposted':datesposted}
df=pd.DataFrame(dict)
df

In [ ]:
df.to_csv('indeed-time.csv', index=False, encoding='utf-8')

In [ ]:
jobs =[]
for div in soup.find_all('div', attrs={'class':'jobsearch-SerpJobCard unifiedRow row result'}):
  for a in div.find_all('a', attrs={'class':'jobtitle turnstileLink'}):
    print(a["title"])


Full set

In [ ]:
!pip install jobs_scraper

  Created wheel for jobs-scraper: filename=jobs_scraper-0.0.10-cp37-none-any.whl size=4849 sha256=794e507054c7efea080296534d2cd5a19e99dada9b5205145b93b9b8190019ce
  Stored in directory: /root/.cache/pip/wheels/86/dd/7f/7afacdf410d538af509942447b53d3b74b390da2793038d745
Successfully built jobs-scraper


In [ ]:
from jobs_scraper import JobsScraper
from datetime import datetime
scraper = JobsScraper(country="us", position="fintech", location="United+States", pages=100)
df = scraper.scrape()

Scraping CUSTOMER SERVICE REPRESENTATIVE...
Scraping Director, Software Development FinTech...
Scraping Corporate M&A Associate...
Scraping Head of FinTech Customer Success...
Scraping Senior Investment Officer...
Scraping Financial Analyst...
Scraping FinTech Associate — Mid...
Scraping Financial Analyst...
Scraping Customer Success Support Specialist - US...
Scraping Investment Banking - Financial Institutions Group - FinTech,......
Scraping Chief of Staff...
Scraping Analyst, Fintech Products and Programs...
Scraping Customer Success...
Scraping Software Development Manager - FinTech...
Scraping DeFi analyst - Remote (Europe)...
Scraping Customer Success...
Scraping Software Development Manager - FinTech...
Scraping DeFi analyst - Remote (Europe)...
Scraping AML/BSA Compliance Analyst...
Scraping Fintech Compliance Manager...
Scraping Entry Level Data Analyst - 4487...
Scraping Manager/Sr. Manager – Fintech Partnerships...
Scraping IT Application Engineering Manager - FinTech...
Scr

# Analysis using words from indeed 

In [ ]:
# load the necessary libraries
import codecs
import json
import matplotlib.pyplot as plt
import pandas as pd
import requests
import re
import time
import warnings
from bs4 import BeautifulSoup
from collections import Counter
from nltk.corpus import stopwords
from pandas import DataFrame
from tqdm import trange
from sqlalchemy import create_engine
import ast, json
from nltk.corpus import stopwords

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:

# This saves letters, and the numbers 
delete_matching = "[^a-zA-Z0-9]"  

lines = (line.strip() for line in sf_job_description_da.splitlines())

# Break multi-headlines into a line each
chunks = (phrase.strip() for line in lines for phrase in line.split("  "))

# Remove all blank lines and ends of line
text = ''.join(chunk + ' ' for chunk in chunks if chunk).encode('utf-8')

# Clean all of the unicode
text = text.decode('unicode_escape')

# Remove any terms that aren't words
text = re.sub(delete_matching, " ", text)

# Lower case everything and split them apart
text = text.lower().split()  

# Set up stop words to remove the stop words 
stop_words = set(stopwords.words("english"))

# Save all words in a list
words = list()
for w in text:
    if w not in stop_words and len(w) > 0:
        words.append(w)

print(words)

['0', 'proprietary', 'platform', 'paynuver', 'clients', 'easily', 'create', 'secure', 'cards', 'online', 'payment', 'accounts', 'disburses', 'funds', 'payee', '1', 'minimum', 'educational', 'background', 'includes', 'bachelor', 'degree', 'computer', 'science', 'related', 'technical', 'field', '2', 'experience', 'financial', 'services', 'fintech', 'industry', 'strongly', 'preferred', 'addition', 'experience', 'attorneys', 'specialized', 'sub', 'practices', 'within', '3', '15', 'years', 'experience', 'b2b', 'customer', 'service', 'support', 'roles', 'ideally', 'within', 'fintech', 'banking', 'payments', 'financial', 'services', 'organizations', '4', 'financial', 'institutions', 'group', 'fig', 'one', 'major', 'industry', 'groups', 'ifc', 'responsible', 'managing', 'roughly', '40', 'ifc', 'overall', 'business', 'terms', '5', '5', 'years', 'experience', 'firm', 'technology', 'including', 'fintech', 'finance', 'banking', 'credit', 'card', 'industry', 'generate', 'recurring', 'ad', 'hoc', 'r

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: invalid escape sequence '\/'
  


In [ ]:
# Count the frequency
freqs = Counter(words)
freqs

Counter({'0': 2,
         'proprietary': 3,
         'platform': 24,
         'paynuver': 2,
         'clients': 24,
         'easily': 3,
         'create': 12,
         'secure': 5,
         'cards': 5,
         'online': 11,
         'payment': 28,
         'accounts': 14,
         'disburses': 2,
         'funds': 4,
         'payee': 2,
         '1': 19,
         'minimum': 9,
         'educational': 1,
         'background': 13,
         'includes': 8,
         'bachelor': 7,
         'degree': 18,
         'computer': 11,
         'science': 11,
         'related': 14,
         'technical': 31,
         'field': 8,
         '2': 26,
         'experience': 215,
         'financial': 106,
         'services': 63,
         'fintech': 182,
         'industry': 47,
         'strongly': 3,
         'preferred': 27,
         'addition': 2,
         'attorneys': 2,
         'specialized': 1,
         'sub': 1,
         'practices': 2,
         'within': 29,
         '3': 24,
         '1

In [ ]:
out_dict = dict([(x, freqs[x.lower()]) for x in text])
top_words_sf = pd.DataFrame.from_dict(out_dict, orient='index', columns=['Frequency']).reset_index()
top_words_sf = top_words_sf.rename(columns={'index': 'Keyword'}).sort_values(by='Frequency', ascending=False).reset_index(drop=True)
top_words_sf = top_words_sf.loc[top_words_sf['Frequency'] > 0]
top_words_sf

,Keyword,Frequency
0,experience,215
1,fintech,182
2,financial,106
3,product,99
4,years,95
...,...,...
3406,295,1
3407,314,1
3408,294,1
3409,313,1


In [ ]:
top_words_sf.to_csv('topwords.csv', index=False, encoding='utf-8')

In [ ]:
analyst_skills = ['R', 'Python', 'Excel', 'JavaScript', 'Tableau', 'Looker', 'SQL', 'NoSQL', 'PostgreSQL', 'MySql', 'MongoDB', 'Visualization', 'Statistics', 'Analysis', 'Communication', 'Storytelling', 'Prediction', 'ML', 'Wrangling', 'Scripting', 'Scala', 'AWS', 'SAS', 'Oracle', 'Spark', 'Cloud', 'Hadoop', 'Java']

In [ ]:
keywords_lower = [x.lower() for x in analyst_skills]
# Save all words in a list
words2 = list()
for w in text:
    if w not in keywords_lower:
        w = w.replace('.', '')
    if w not in stop_words and len(w) > 0:
        words2.append(w)

print(words2)

['0', 'proprietary', 'platform', 'paynuver', 'clients', 'easily', 'create', 'secure', 'cards', 'online', 'payment', 'accounts', 'disburses', 'funds', 'payee', '1', 'minimum', 'educational', 'background', 'includes', 'bachelor', 'degree', 'computer', 'science', 'related', 'technical', 'field', '2', 'experience', 'financial', 'services', 'fintech', 'industry', 'strongly', 'preferred', 'addition', 'experience', 'attorneys', 'specialized', 'sub', 'practices', 'within', '3', '15', 'years', 'experience', 'b2b', 'customer', 'service', 'support', 'roles', 'ideally', 'within', 'fintech', 'banking', 'payments', 'financial', 'services', 'organizations', '4', 'financial', 'institutions', 'group', 'fig', 'one', 'major', 'industry', 'groups', 'ifc', 'responsible', 'managing', 'roughly', '40', 'ifc', 'overall', 'business', 'terms', '5', '5', 'years', 'experience', 'firm', 'technology', 'including', 'fintech', 'finance', 'banking', 'credit', 'card', 'industry', 'generate', 'recurring', 'ad', 'hoc', 'r

In [ ]:
# Count the frequency
freqs = Counter(words)

# What are the top keywords
out_dict = dict([(x, freqs[x.lower()]) for x in analyst_skills])
top_words_sf = pd.DataFrame.from_dict(out_dict, orient='index', columns=['Frequency']).reset_index()
top_words_sf = top_words_sf.rename(columns={'index': 'Keyword'}).sort_values(by='Frequency', ascending=False).reset_index(drop=True)
top_words_sf = top_words_sf.loc[top_words_sf['Frequency'] > 0]
top_words_sf

,Keyword,Frequency
0,Analysis,42
1,Communication,24
2,Cloud,24
3,Python,16
4,JavaScript,14
5,Java,10
6,SQL,8
7,AWS,8
8,Statistics,4
9,Excel,4
